 # BEM coupling with plane wave approximation

In [18]:
import capytaine as cpt
import numpy as np
import matplotlib.pyplot as plt
from capytaine.bem.airy_waves import airy_waves_potential, airy_waves_velocity, froude_krylov_force
from capytaine.bem.solver import BEMSolver
from capytaine.green_functions.delhommeau import Delhommeau
from capytaine.bem.engines import BasicMatrixEngine
from capytaine.bem.problems_and_results import RadiationProblem
from capytaine.bodies.predefined.spheres import Sphere
cpt.__version__ #get_potential_on_mesh if not version 2.0


p = 0
omega = 1.1
rho = 850 # density of our special material
wave_amp = 1
wave_num =  1.0/9.81


### Some functions that are required down the road for implementation

In [19]:
#%%writefile pwa_utils.py
def generate_body(xyz):
    mesh1 = cpt.meshes.predefined.mesh_sphere(radius=2,center=(xyz[0],xyz[1],xyz[2]))
    body = cpt.FloatingBody(mesh1)
    body.add_translation_dof(name='Heave')
    body = body.immersed_part()
    body.name = f'{xyz[0]}_{xyz[1]}_{xyz[2]}'
    return body


def get_results(problems):
    results = [solver.solve(pb, keep_details = True) for pb in sorted(problems)]
    return results


#calculate angle theta_ij from centre of one body to other
def theta_ij(X,Y): 
    x1,y1= X[0],X[1]
    x2,y2 = Y[0], Y[1]
    
    if x1 ==x2 and y1==y2:
        return 0
    if x2==x1:
        theta = np.pi/2
    else:
        theta = np.arctan((y2-y1)/(x2-x1))
    return theta


#step 2
def phi_j_star(phi_ij,theta,X,Y,z,k):
    
    '''phi_ij is the vector of all the effect at that body from all other bodies'''
    x,y = X[0],X[1]
    xj,yj = Y[0],Y[1]
    if x==xj and y==yj:
        return 0
    multiplier = np.exp((1j*k*(-1*np.abs(x-xj))*np.cos(theta)) + ((-1*np.abs(y-yj))*np.sin(theta)))
   # print(f'the multiplier for {X}{Y} and {theta} is {multiplier}')
    res = phi_ij * multiplier #kz = 0 #e^kz = 1
    return res

#{(10, 10, 0): {(10, 10, 0): 0,
  #(0, 0, 0): (8.415476709952118-2.9519008598532284j),
  #(5, 5, 0): (8.415476709952118-2.9519008598532284j),
  #(15, 15, 0): (8.415476709952118-2.9519008598532284j)

def get_phistarj_sum(phi_starj,xyzees):
    xyz_phi = {xyz :[] for xyz in xyzees}
    for k,v in phi_starj.items():
        for s,m in v.items():
          #  print(f"thee value of m = {m}")
            xyz_phi[k].append(m)
     #   print("next xyz")
    print(xyz_phi.items())
    xyz_phi = {k:sum(v) for k,v in xyz_phi.items()}
    #print("Afteer summation ")
   # print(xyz_phi.items())
    return xyz_phi


The method is based upon an idea due to Simon 3
originally devised in connection with the theory of arrays
of wave-power devices. A diverging wave scattered from
one cylinder is replaced by a plane wave of appropriate
amplitude in the neighbourhood of another cylinder. Once
the amplitude and phase of the equivalent plane wave have
been determined, **the problem reduces to summing the
effects of plane waves on any given cylinder**

#### 1. Initialize the bodies and define diffraction and radiation problems and solve

This step is performed in isolation and this is the first BEM run where we evaluate the green's function

In [20]:
from capytaine import assemble_dataset
#xyzees = {(0,0,0),(5,5,0),(10,10,0),(15,15,0)}
xyzees = {(0,0,0),(15000000,15000000,0)}
    
bodies = [generate_body(xyz) for xyz in xyzees ]

#neighbors = {(0,0,0):[(5,5,0),(10,10,0),(15,15,0)],  #so bad..need to write a funky func for it
 #           (10,10,0):[(0,0,0),(5,5,0),(15,15,0)],
  #           (5,5,0):[(0,0,0),(10,10,0),(15,15,0)],
   #          (15,15,0):[(0,0,0),(10,10,0),(5,5,0)]     
    #        }


neighbors = {(15000000,15000000,0):(0,0,0),(0,0,0):(15000000,15000000,0)}

def get_neighbors(xyzees):
    neighbor = {xyz:[] for xyz in xyzees}
    for xyz in xyzees:
        for zyx in xyzees:
            if not xyz == zyx:
                neighbor[xyz].append(zyx)
    return neighbor


print(get_neighbors(xyzees))


loc_bodies = {body:xyz for xyz,body in zip(xyzees,bodies)}
loc_to_body = {xyz:body for xyz,body in zip(xyzees,bodies)}
#caching interaction matrices help make things go brrrrrrrr

cache_engine = BasicMatrixEngine(matrix_cache_size=1)

solver = cpt.BEMSolver(engine=cache_engine)


diff_problems = {body:cpt.DiffractionProblem(body=body, water_depth = np.infty,
                                      omega=omega, wave_direction=0.) for body in bodies}

diff_loc= {generate_body(loc):loc for loc in xyzees }

loc_diff = {loc_bodies.get(body):diff for body,diff in diff_problems.items() }

rad_problems = {body: cpt.RadiationProblem(body=body, water_depth = np.infty,
                                      omega=omega) for body in bodies}

diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}
rad_results = {body:solver.solve(problem) for body,problem in rad_problems.items()}


diff_results_first_iter = diff_results
rad_results_first_iter = rad_results
#added_mass = {body:assemble_dataset(result) for body, result in rad_results.items()}



{(15000000, 15000000, 0): [(0, 0, 0)], (0, 0, 0): [(15000000, 15000000, 0)]}


In [21]:
#diff problem round 2 
def update_boundary_condition(DiffProblem,new_boundary_excitation):
    DiffProblem.boundary_condition +=new_boundary_excitation
    return DiffProblem
# {body:update_boundary_condition(cpt.DiffractionProblem(body=body, water_depth = np.infty,
#                                       omega=omega, wave_direction=0.), 0.2) for body in bodies}


### 2. Get the Incident potentials, diffraction potentials, radiation potentials for each body at their own location and the location of other bodies


STEPS

1) Assume each body as isolated and solve the diffraction and radiation problem with $usual$ body condition
2) Find the potential of the incident PLANE WAVE.$A_{ij}$ being the apmplitude of body $j$ at body $i$, this can be generated at any other location from capytaine. And according to simon(1982) we can approximate the impact of outgoing waves from a body on all other bodies by an incident plane wave of appropriately chosen amplitude $A_{ij}$.

3) Find the total effect at each body j due to other body i
4) The diffraction potential depends on the boundary condition from the incoming incident plane waves so that has to be updated for new iteration.
5)  Having computed the effect at all the bodies in the array we compute the contribution of all the bodies as isolated (step 1) due to the excitation induced by new φ∗
    1) New excitation means new boundary condition. CALCULATE NEW BOUNDARY CONDITION for each iteration until convergence. Is new boundary condition  = usual boundary condition + new $\phi^*$?
    2) For each body to calculate new diffraction and radiation problem ? Meaning either re-evaluate the influence matrices or store from last time (step 1) if the matrix do not change themselves.
    3) Use the already stored influence matrices (green's function matrix and its normal derivative matrix)
   



#### find the location of neighbors to each floating body

In [22]:
body_neighbors_locs = {body:neighbors.get(loc_bodies.get(body)) for body in bodies}
body_neighbors_locs

{FloatingBody(mesh=sphere_24, dofs={Heave}, name=15000000_15000000_0): (0,
  0,
  0),
 FloatingBody(mesh=sphere_30, dofs={Heave}, name=0_0_0): (15000000,
  15000000,
  0)}

###  Get the potential of a body to other location - Find the potential of incident PLANE WAVE $A_{IJ}$

> Capytaine version 1.5 has the function as `get_potential_on_mesh` while new v2.0 have `compute_potential`

In [23]:
body_potential_at_neighbors = {body:dict(zip(body_neighbors_locs[body], 
                                      solver.compute_potential(np.array(body_neighbors_locs[body]),diff_results[body]))) for body in bodies}
body_potential_at_neighbors

{FloatingBody(mesh=sphere_24, dofs={Heave}, name=15000000_15000000_0): {0: (0.0003390139419528805+0.0001298202123125508j)},
 FloatingBody(mesh=sphere_30, dofs={Heave}, name=0_0_0): {15000000: (0.00022358294664560224+0.00011046447632589434j)}}

### $\phi_{ij}$ or  $𝐴_{ij}$ Get the all other potential at each location/body..


This is just re-structuring earlier result so that we have results of all other WECs at each location

In [24]:
# def get_all_other_phi(body_potential_at_neighbors):
all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
all_other_phi_each_loc

{(15000000, 15000000, 0): {(15000000, 15000000, 0): 0, (0, 0, 0): 0},
 (0, 0, 0): {(15000000, 15000000, 0): 0, (0, 0, 0): 0}}

### $\phi_j^*$ get the total effect of each bodies

In [25]:
#First fet the thetas

thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
thetas

{(15000000, 15000000, 0): {(15000000, 15000000, 0): 0,
  (0, 0, 0): 0.7853981633974483},
 (0, 0, 0): {(15000000, 15000000, 0): 0.7853981633974483, (0, 0, 0): 0}}

In [26]:
z = 0
phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
phi_starj

{(15000000, 15000000, 0): {(15000000, 15000000, 0): 0, (0, 0, 0): 0j},
 (0, 0, 0): {(15000000, 15000000, 0): 0j, (0, 0, 0): 0}}

### Get the Total potential on the body surface - 
#### Make sure we add all N but not when i==j so the value should be 0 for same location 

In [27]:

new_excitation = get_phistarj_sum(phi_starj,xyzees)
new_excitation     

dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])


{(15000000, 15000000, 0): 0j, (0, 0, 0): 0j}

Having computed this effect at each body in the array we then compute the contribution of all the bodies as isolated ( as in step 1) induced by new excitation

#### Get the all other potential at each location/body with new excitation
If new excitation is the potential and we have this new excitation **at** each location, how do we go from getting the contribution of this $\phi$ to other location?

- New boundary condition !!!!! and solve the diffraction and radiation problems with new boundary condition. 
 - but how does the new excitation give new boundary condition?

- Write a function to get the influence matrices and store them in a dictionary 

- Calculate new `body_potential_at_neighbors` from earlier to loop it back in a while loop. The potential at other location this time is not going to be the airywave potential calculation but just the  calculation function `phi_j_star` but keep new potential at each neigbors in the format `body_potential_at_neighbors` 



In [28]:
def update_boundary_condition(DiffProblem,new_boundary_excitation):
    DiffProblem.boundary_condition +=new_boundary_excitation
    return DiffProblem

### Updated diffraction and radiation problems

In [29]:
# def get_influence_matrices(prob):
#             S, K = self.engine.build_matrices(
#             problem.body.mesh, problem.body.mesh,
#             problem.free_surface, problem.water_depth, problem.wavenumber,
#             self.green_function
#         )
        
#         return S,K


diff_problems = {body:update_boundary_condition(cpt.DiffractionProblem(body=body, water_depth = np.infty,
                                      omega=omega, wave_direction=0.), new_excitation[loc_bodies[body]]) for body in bodies}
 
# think about updating rad_problems too
rad_problems = {body: cpt.RadiationProblem(body=body, water_depth = np.infty,
                                      omega=omega) for body in bodies}

> Now we solve the updated diffraction problem and radiation problem - (back to step1)


### Now we basically while loop the whole thing until the max-iteration

In [30]:
N_bodies = 4
max_iteration = 2*N_bodies #(dead or alive lol)

# body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
#                                        airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}


iterate = 0
while iterate<=max_iteration:
    #go back to step 1 and solve the diff and radiation problems again
    #if it is slow it is running green's function again so we may need to cache it. now let's just go through with it
    diff_problems = {body:update_boundary_condition(cpt.DiffractionProblem(body=body, water_depth = np.infty,
                                      omega=omega, wave_direction=0.), new_excitation[loc_bodies[body]]) for body in bodies}
 
    rad_problems = {body: cpt.RadiationProblem(body=body, water_depth = np.infty,
                                      omega=omega) for body in bodies}
    
    diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}
    rad_results = {body:solver.solve(problem) for body,problem in rad_problems.items()}
    body_potential_at_neighbors = {body:dict(zip(body_neighbors_locs[body], 
                                      solver.compute_potential(np.array(body_neighbors_locs[body]),diff_results[body]))) for body in bodies}
    all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
    phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
    new_excitation = get_phistarj_sum(phi_starj,xyzees)
    iterate+=1


dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])
dict_items([((15000000, 15000000, 0), [0, 0j]), ((0, 0, 0), [0j, 0])])


In [31]:
new_potential = new_excitation

In [32]:
# # function to input new potential from plane-wave approximation
# diff_problems = {body:DiffractionProblemIter(body=body, sea_bottom=-np.infty,
#                                       omega=omega, wave_direction=0.) for body in bodies}

# diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}


def solve(diff,diff_res, rad_res,new_potential,keep_details=True):
        """Solve the linear potential flow problem.
        Parameters
        ----------
        problem: LinearPotentialFlowProblem
            the problem to be solved
        keep_details: bool, optional
            if True, store the sources and the potential on the floating body in the output object
            (default: True)
        Returns
        -------
        LinearPotentialFlowResult
            an object storing the problem data and its results
        """
        
        diff_pot = diff_res.potential
        print(diff)
       
        rad_pot = rad_res.potential
        potential = new_potential + diff_pot + rad_pot
        rho = 1000
        new_pressure = rho * potential
        # Actually, for diffraction problems: pressure over jω
        #           for radiation problems:   pressure over -ω²
        # The correction is done in `store_force` in the `result` object.

        new_forces = diff.body.integrate_pressure(new_pressure)
        

        if not keep_details:
            results = diff.make_results_container(new_forces)
        else:
            results = diff.make_results_container(new_forces, diff_res.sources, potential, new_pressure)
            
        added_mass = np.abs(new_forces['Heave'].real)
        
        damping = np.abs(new_forces['Heave'].imag) * diff.omega #abs because damping should be positive? does not make sense
        return new_forces, {'added_mass':added_mass}, {'damping':damping}
    

    


In [33]:


def solve_forces(diff,diff_res, rad_res,keep_details=True):
    
    """
    """
    diff_pot = diff_res.potential

    rad_pot = rad_res.potential
    potential =  diff_pot + rad_pot
    rho = 1000
    new_pressure = rho * potential
    # Actually, for diffraction problems: pressure over jω
    #           for radiation problems:   pressure over -ω²
    # The correction is done in `store_force` in the `result` object.

    new_forces = diff.body.integrate_pressure(new_pressure)

    #         if not keep_details:
    #             result = problem.make_results_container(new_forces)
    #         else:
    #             result = problem.make_results_container(new_forces, sources, new_potential, new_pressure)
    return new_forces

new_results = {loc_to_body.get(loc):solve_forces(diff_prob,diff_results[loc_to_body.get(loc)],
                                                 rad_results[loc_to_body.get(loc)]) for loc,diff_prob in loc_diff.items()}
new_results

{FloatingBody(mesh=sphere_24, dofs={Heave}, name=15000000_15000000_0): {'Heave': (1581.241720675112+49.6873182720709j)},
 FloatingBody(mesh=sphere_30, dofs={Heave}, name=0_0_0): {'Heave': (-51.97723490949254-504.30275454724193j)}}

Comparing that to the original results

In [34]:
starting_results = {loc_to_body.get(loc):solve_forces(diff_prob,diff_results_first_iter[loc_to_body.get(loc)],
                                                 rad_results_first_iter[loc_to_body.get(loc)]) for loc,diff_prob in loc_diff.items()}
starting_results

{FloatingBody(mesh=sphere_24, dofs={Heave}, name=15000000_15000000_0): {'Heave': (1581.241720675112+49.6873182720709j)},
 FloatingBody(mesh=sphere_30, dofs={Heave}, name=0_0_0): {'Heave': (-51.97723490949254-504.30275454724193j)}}

Okay confirmed..that this gives the correct result if two bodies are very far away!